In [1]:
try: 
    sc.stop()
except: 
    pass
import os
import sys
import datetime
import pandas as pd
from datetime import date
os.environ['SPARK_HOME']            = '/opt/cloudera/parcels/SPARK2/lib/spark2'
os.environ['PYSPARK_DRIVER_PYTHON'] = '/opt/cloudera/parcels/PYENV.ZNO20008661/bin/python'
os.environ['PYSPARK_PYTHON']        = '/opt/cloudera/parcels/PYENV.ZNO20008661/bin/python'
os.environ['LD_LIBRARY_PATH']       = '/opt/python/virtualenv/jupyter/lib'
sys.path.insert(0, '/opt/cloudera/parcels/SPARK2/lib/spark2/python')
sys.path.insert(0, '/opt/cloudera/parcels/SPARK2/lib/spark2/python/lib/py4j-0.10.7-src.zip')
pd.set_option('max_colwidth',1200)
from pyspark.sql import functions as func
from pyspark import SparkContext, SparkConf, HiveContext, SQLContext

# модули под задачу 
import json 
import time
from tqdm import tqdm

# full - жирный динамический - запуск желателен только при полупустом кластере
conf = SparkConf().setAppName('korneev_education_spark2')\
                  .setMaster("yarn-client")\
                  .set('spark.ui.port','8175')\
                  .set('spark.driver.maxResultSize', '20G')\
                  .set('spark.driver.memory', '16G')\
                  .set('spark.yarn.executor.memoryOverhead', '3G')\
                  .set('spark.executor.memory', '20G')\
                  .set('spark.executor.cores', '10')\
                  .set('spark.dynamicAllocation.maxExecutors', '10')\
                  .set('spark.dynamicAllocation.enabled', 'true')\
                  .set('spark.shuffle.service.enabled', 'true')\
                  .set('spark.dynamicAllocation.cachedExecutorIdleTimeout', '600s')\
                  .set("spark.sql.legacy.allowCreatingManagedTableUsingNonemptyLocation","true")\
                  .set('spark.kryoserializer.buffer.max','512m')\
                  .set('spark.rpc.message.maxSize','1536')

sc = SparkContext.getOrCreate(conf=conf)
sqlContext = SQLContext(sc)
spark = sqlContext.sparkSession
spark.sql('set mapred.input.dir.recursive = true')
spark.sql('set spark.sql.parquet.binaryAsString = true')
spark.sql('set spark.sql.hive.convertMetastoreParquet = false')
sc 

<SparkContext master=yarn-client appName=korneev_util>

In [93]:
# film for calculation: 302 
# L.A. Confidential (1997) 

In [50]:
src = pd.read_csv('u.data', sep='\t', header=None).rename(columns={0:'userid', 1:'itemid', 2:'rate', 3:'timestampcolumn'})
src.sample()

,userid,itemid,rate,timestampcolumn
95497,894,628,3,880416102


In [60]:
df = spark.createDataFrame(src)[['itemid', 'rate']]
df

DataFrame[itemid: bigint, rate: bigint]

In [61]:
df = df.repartition(10)
df.rdd.getNumPartitions() 

10

In [81]:
hist_all = df[['rate']].groupBy('rate').count().toPandas().sort_values( by=['rate'])['count'].tolist()
hist_film = df[df['itemid']=='302'][['rate']].groupBy('rate').count().toPandas().sort_values( by=['rate'])['count'].tolist()

In [82]:
hist_all

[6110, 11370, 27145, 34174, 21201]

In [83]:
hist_film

[2, 10, 46, 119, 120]

In [87]:
result = {'hist_all': hist_all, 'hist_film':hist_film}

In [92]:
import json 
result = {'hist_all': hist_all, 'hist_film':hist_film}
with open('result.json', 'w') as f: 
    json.dump(result, f) 

In [66]:
df[df['itemid']=='302'][['rate']].groupBy('rate').count()

[Row(rate=5, count=120),
 Row(rate=1, count=2),
 Row(rate=3, count=46),
 Row(rate=2, count=10),
 Row(rate=4, count=119)]